<a href="https://colab.research.google.com/github/nerudxlf/only_wos_or_scopus/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re
import pandas as pd

from google.colab import files

files.upload()

In [12]:
global result_full_data
choice = 2  # Если Scopus то установить в 0, если WoS то в 1, если и WoS и Scopus то 3

wos_data = pd.read_excel("AllPublin2020.xls")
scopus_data = pd.read_csv("ExpotrScopus.csv")
if choice == 1 or choice == 2:
  wos_data_filter = wos_data.filter(items=['Article Title'])
  scopus_data_filter = scopus_data.filter(items=[' Title'])
  wos_data_filter['KEY'] = wos_data_filter.apply(
      lambda x: re.compile('\W').sub('', x.get("Article Title")).upper(), axis=1)
  scopus_data_filter['KEY'] = scopus_data_filter.apply(
      lambda x: re.compile('\W').sub('', x.get(" Title")).upper(), axis=1)
  wos_data_filter_key = wos_data_filter.filter(items=['KEY'])
  scopus_data_filter_key = scopus_data_filter.filter(items=['KEY'])
  result_wos_scopus = pd.concat([wos_data_filter_key, scopus_data_filter_key])
  result_wos_scopus = result_wos_scopus.loc[result_wos_scopus.duplicated(keep=False), :]
  if choice == 1:
    concat_data = pd.concat([wos_data_filter_key, result_wos_scopus])
    result_data_key = concat_data.drop_duplicates(keep=False)
    result_data_key_title = pd.merge(left=result_data_key, right=wos_data_filter, left_on="KEY", right_on="KEY")
    result_full_data = pd.merge(left=result_data_key_title, right=wos_data, left_on="Article Title",
          right_on="Article Title")
  elif choice == 2:
    concat_data = pd.concat([scopus_data_filter_key, result_wos_scopus])
    result_data_key = concat_data.drop_duplicates(keep=False)
    result_data_key_title = pd.merge(left=result_data_key, right=scopus_data_filter, left_on="KEY",
          right_on="KEY")
    result_full_data = pd.merge(left=result_data_key_title, right=scopus_data, left_on=" Title",
          right_on=" Title")
elif choice == 3:
  # в список items можно добавить любые столбцы для выделения из таблицы
  wos_data_filter = wos_data.filter(
      items=['Publication Type', 'Authors', 'Article Title', 'Publication Year', 'Book Series Title',
            'Research Areas'])
  scopus_data_filter = scopus_data.filter(items=[' Title', 'Authors', ' Year'])
  wos_data_filter['KEY'] = wos_data_filter.apply(
      lambda x: re.compile('\W').sub('', x.get("Article Title")).upper(), axis=1)
  scopus_data_filter['KEY'] = scopus_data_filter.apply(
      lambda x: re.compile('\W').sub('', x.get(" Title")).upper(), axis=1)
  result_full_data = pd.merge(left=wos_data_filter, right=scopus_data_filter, left_on="KEY", right_on="KEY")

result_full_data.to_excel("result_data.xlsx", index=False)